In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from PIL import Image, ImageDraw
import requests
from io import BytesIO
import os
import pandas as pd

from notebooks_util import * 

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Loading the Datasets

In [2]:
cifar_10_dir = 'data/cifar-10-batches-py'

train_data, train_filenames, train_labels, test_data, test_filenames, test_labels, label_names = \
    load_cifar_10_data(cifar_10_dir)

test_data_ = test_data
np.random.shuffle(test_data_)
test_data_ = np.repeat(test_data_, 7, axis=1)
test_data_ = np.repeat(test_data_, 7, axis=2)

In [4]:
test_data_2 = np.load('data/cifar10.1_v6_data.npy')
np.random.shuffle(test_data_2)
test_data_2_ = test_data_2
test_data_2_ = np.repeat(test_data_2_, 7, axis=1)
test_data_2_ = np.repeat(test_data_2_, 7, axis=2)

In [29]:
features = np.concatenate([test_data_, test_data_2_])

In [9]:
labels = np.concatenate([np.zeros((len(test_data_), )), np.ones((len(test_data_2_), ))])

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)

# Base Model

In [18]:
vgg = vgg16.VGG16(weights='imagenet', include_top=False)
vgg.trainable = False

In [19]:
vgg.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [21]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1)
model = tf.keras.Sequential([
  vgg,
  global_average_layer,
  prediction_layer
])

In [22]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


# Train the Model

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    validation_data=(X_test, y_test))

Train on 8040 samples, validate on 3960 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
  32/8040 [..............................] - ETA: 57:39 - loss: 7.7118 - acc: 0.4062